# Lesson 5 REST Demo

## Retrieving Information From Web Pages

You may occasionally need to GET the contents of web pages in order to “scrape” text information from sites that wouldn’t otherwise be accessible via an API.  Fortunately, the `requests` library is equally useful for retrieving the contents of web pages as it is for accessing APIs.

One example use case is performing sentiment analysis on messages in online forums; you could GET the content of pages on a particular topic, use string methods to extract relevant information, and then either use that information to train a model or provide that information to an LLM for analysis.

### GETting a Web Page From a URL

Let’s start with the kind of GET request that billions of people use every day: a request for the contents of a web page. In this case, we’ll use a GET request to retrieve the contents of Kodeco’s home page.

In [ ]:
import requests

response = requests.get("https://kodeco.com/")

if response.status_code == 200:
    print("Successfully retrieved the web page!")
    print(f"Status code: {response.status_code}.")
    print(f"Content type: {response.headers["Content-Type"]}.")
    print("=====")
    print(response.text)
else:
    print(f"Failed to retrieve the web page. Status code: {response.status_code}.")

This code sends a GET request to Kodeco’s server for the resource located at the URL `https://kodeco.com/`, which doesn’t include a pathname or filename. Since the URL ends with a directory and not a filename, it returns the default resource located at `kodeco.com`, the HTML file defining Kodeco’s home page.

Because a resource exists at `kodeco.com`, the server can fulfill the GET request made by the code. As a result, the server’s response includes the status code `200`, meaning “OK” and indicating success. This value is contained in the response object’s `status_code` property.

Although we know that `kodeco.com` is the location of Kodeco’s home page, the code confirms that the resource is HTML data. It does this by accessing the response object’s `headers` property, which contains the headers of the response in Python dictionary form. The `Content-Type` header — the value corresponding to `headers`’ `Content-Type` key — for this request is `text/html; charset=utf-8`, indicating that the response is HTML content using the UTF-8 encoding standard, which supports a wide array of characters.

Since the response contains HTML data, we use the response’s `text` property to access the response body, which returns it as a string. The code then displays this string.

Your browser executes a similar GET request when you point it at `kodeco.com`; the difference is that it renders the HTML as a web page instead of simply displaying it.

### GETting a Web Page From an Invalid URL

What happens when you try to GET from a URL that doesn’t correspond to a web resource? You can find out by changing the URL in the previous code to one for a resource that doesn’t exist.

Update the definition of the `response` variable in the previous code to the following, then run the cell again:

```python
response = requests.get("https://kodeco.com/non-existent-page")
```

In [ ]:
response = requests.get("https://kodeco.com/non-existent-page")

if response.status_code == 200:
    print("Successfully retrieved the web page!")
    print(f"Status code: {response.status_code}.")
    print(f"Content type: {response.headers["Content-Type"]}.")
    print("=====")
    print(response.text)
else:
    print(f"Failed to retrieve the web page. Status code: {response.status_code}.")

This time, since the code is now attempting to GET a web resource that doesn’t exist, the server’s response includes the well-known status code `404`, which means “Not found.”

In the case of a web page, there may still be some content included in the response. Many websites, Kodeco’s included, present the user with a “404” page when they try to navigate to a URL that doesn’t have a corresponding web page. You’ll find the HTML for Kodeco’s “404” page in the `text` property of the response object.

## Retrieving Information From REST APIs

### Making the Simplest Possible GET Request

The _Star Wars_ API is a fun API to play with. It’s an API that acts as an encyclopedia of _Star Wars_ people, species, planets, vehicles, spaceships, and films (or at least Episodes 1 through 7). You don’t need to pay or register for an API key to use it — you can simply start making calls to it. The API has a website at [`swapi.dev`](https://swapi.dev/) and its base URL is [`swapi.dev/api`](https://swapi.dev/api/).

This API is a rather friendly design. If you make a GET request to the base URL, you’ll receive a JSON object listing all the API endpoints.

Make this call:

In [ ]:
import json

SWAPI_BASE_URL = "https://swapi.dev/api/"
response = requests.get(SWAPI_BASE_URL)

if response.status_code == 200:
    data = response.json()
    pretty_json = json.dumps(data, indent=4, sort_keys=True)
    print(pretty_json)

When you run the code above, you should see a dictionary listing the _Star Wars_ API’s endpoints, where the keys are the names of the endpoints and the values are the corresponding endpoint URLs. You’ll use one of these URLs shortly.

Since we’re calling an API rather than retrieving a web page’s contents, the code uses the response object’s `json()` method instead of its `text` property. The `json()` method automatically converts the JSON text in the response into Python lists and dictionaries, from which you can extract the data you need.

To display the retrieved JSON data in a readable format, the code uses the `json.dumps()` method to convert it into a string representation. Using the optional `indent` parameter makes `json.dumps()` format the string using one line per list or dictionary element, using the specified number of spaces for each level of indentation. Setting the optional `sort_keys` parameter to `True` causes `json.dumps()` to present dictionary keys in ascending alphabetical order.

### Making a GET Request With Path Parameters

Let’s use one of the _Star Wars_ API endpoints: the “people” endpoint.

The _Star Wars_ API uses path parameters, which means that you make requests by appending the ID of the resource you want to the end of the URL for the endpint for that resource. In the case of people, you append the ID of the person whose information you want to the “people” endpoint and use the resulting URL for a GET request.

You’ve probably already guessed who the _Star Wars_ person with the ID of `1` is. You can confirm it by running the following in a new code cell:

In [ ]:
PEOPLE_ENDPOINT = "people/"
person_id = 1
response = requests.get(f"{SWAPI_BASE_URL}{PEOPLE_ENDPOINT}{person_id}")

if response.status_code == 200:
    data = response.json()
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)

You should see a big dictionary full of information about the _Star Wars_ person with the ID of `1`: none other than Luke Skywalker himself. The JSON object in the response has been converted into a Python dictionary that contains all sorts of information about the character.

You may have noticed that some of the information about Luke — such as his homeworld — contains one or more URLs. These URLs act as “pointers” to where that particular information is accessible through the API. To get this information, you perform another GET request using these URLs. Run the code below to get information about Luke Skywalker’s homeworld:

In [ ]:
PEOPLE_ENDPOINT = "people/"
person_id = 1
person_response = requests.get(f"{SWAPI_BASE_URL}{PEOPLE_ENDPOINT}{person_id}")

if person_response.status_code == 200:
    person_data = person_response.json()
    homeworld_endpoint = person_data["homeworld"]
    homeworld_response = requests.get(homeworld_endpoint)
    if homeworld_response.status_code == 200:
        homeworld_data = homeworld_response.json()
        pretty_json = json.dumps(homeworld_data, indent=4)
        print(pretty_json)

You should see a dictionary containing information about Luke’s homeworld, Tatooine.

The code above first GETs Luke Skywalker’s information, which is a dictionary. From that dictionary, it takes the value corresponding to the `homeworld` key, which is a string containing the URL for the resource representing Luke’s homeworld. The code GETs this resource and displays it.

### Making a GET Request With Query Parameters

Let’s look at an API that uses query parameters, where you provide parameters in the form of a query string at the end of the endpoint URL. The API in question is the [Open-Meteo](https://open-meteo.com/) weather API (_meteo_ is the French word for “weather”). Like the _Star Wars_ API, the Open-Meteo API is free of charge to use and doesn’t require you to register for an API key.

Suppose we want to get the current weather in London, England. In order to do so, we need to provide Open-Meteo with the following parameters:

<table>
    <tr>
        <td><strong>Parameter</strong></td>
        <td style="text-align:left;"><strong>Description</strong></td>
    </tr>
    <tr>
        <td><code>latitude</code></td>
        <td>
            <p>The location’s latitude. Positive values mean degrees north of the equator, negative values mean degrees south of the equator, and 0 means the equator. For this example, we’ll use the value <code>51.5072</code>.</p>
        </td>
    </tr>
    <tr>
        <td><code>longitude</code></td>
        <td>
            <p>The location’s longitude. Positive values mean degrees east of the prime meridian, negative values mean degrees west of the prime meridian, and 0 means the prime meridian. For this example, we’ll use the value <code>-0.1276</code>.</p>
        </td>
    </tr>
    <tr>
        <td><code>current</code></td>
        <td>
            <p>The weather values for the current weather that you want to retrieve (listed on <a href="https://open-meteo.com/en/docs">Open-Meteo’s API docs page</a>, in the form of a comma-separated list. For this example, we’ll request three values:</p>
            <ul>
                <li><code>weathercode</code>: A number code for the current weather at the requested location.</li>
                <li><code>temperature_2m</code>: The air temperature at the requested location, as measured at a height of 2 meters (about 6 feet) above the ground. This value is in degrees Celsius.</li>
                <li><code>relativehumidity_2m</code>: The relative humidity at the requested location, as measured at a height of 2 meters (about 6 feet) above the ground. This value is expressed as a percentage — between 0 and 100 inclusive.</li>
            </ul>
        </td>
    </tr>
</table>

Open-Meteo’s endpoint URL for the current weather at a given location is `https://api.open-meteo.com/v1/forecast/`. One way to form the URL for the GET request is to build the URL string starting with the endpoint URL, adding a `?` to the end to denote the start of the query parameters, followed by the query parameters. Here’s the URL for our example (you can click on it to see the response):

[`https://api.open-meteo.com/v1/forecast?latitude=51.5072&longitude=-0.1276&current=weathercode,temperature_2m,relativehumidity_2m`](https://api.open-meteo.com/v1/forecast?latitude=51.5072&longitude=-0.1276&current=weathercode,temperature_2m,relativehumidity_2m))

While this approach works, it’s cumbersome and error-prone. It’s much better to send the GET request to the endpoint URL and use the `params` parameter of the `request.get()` method to define the query paramerters:

In [ ]:
WEATHER_ENDPOINT = "https://api.open-meteo.com/v1/forecast"
parameters = {
    "latitude":  51.5072,
    "longitude": -0.1276,
    "current":   "weathercode,temperature_2m,relativehumidity_2m",
}
response = requests.get(WEATHER_ENDPOINT, params=parameters)

if response.status_code == 200:
    data = response.json()
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)

The part of the response that you’re most interested in is the value for the `"current"` key, a dictionary containing the values for the `weathercode`, `temperature_2m`, and `relativehumidity_2m` parameters.

To translate the `weathercode` value into something more meaningful than a number, we’ll use a dictionary as a lookup table:

In [ ]:
WEATHER_CODE_TABLE = {
    0: "clear sky",
    1: "mainly clear",
    2: "partly cloudy",
    3: "overcast",
    45: "fog",
    48: "depositing rime fog",
    51: "light drizzle",
    53: "moderate drizzle",
    55: "dense drizzle",
    56: "light freezing drizzle",
    57: "dense freezing drizzle",
    61: "slight rain",
    63: "moderate rain",
    65: "heavy rain",
    66: "light freezing rain",
    67: "heavy freezing rain",
    71: "slight snow",
    73: "moderate snow",
    75: "heavy snow",
    77: "snow grains",
    80: "light rain showers",
    81: "moderate rain showers",
    82: "violent rain showers",
    85: "slight snow showers",
    86: "heavy snow showers",
    95: "thunderstorm",
    96: "thunderstorm with slight hail",
    99: "thunderstorm with heavy hail",
}

print(f"The weather in London is: {WEATHER_CODE_TABLE[data["current"]["weathercode"]]}")

## Adding, Updating, and Deleting Data In APIs

In this part of the demo, we’ll use [ReqRes](https://reqres.in/), a REST API designed specifically for testing applications that make calls to an API. You can send GET, POST, PUT, PATCH, and DELETE requests to its endpoints, and it will provide a simulated response. As such, it’s perfect for trying out requests other than GET.

ReqRes has a `users` endpoint, located at [`https://reqres.in/api/users/`](https://reqres.in/api/users/), which we’ll use to add, update, and delete resources in the API.

### Adding a Resource

Let’s pretend that we want to add a new user to the database: the creator of Python, Guido van Rossum. We’ll provide his name and programming language as parameters. We’ll do this by making a POST request to create a new resource in the API:

In [ ]:
USERS_ENDPOINT = "https://reqres.in/api/users/"
post_data = {
    "language": "Python",
    "creator":  "Guido van Rossum",
}
response = requests.post(USERS_ENDPOINT, data=post_data)

if response.status_code == 201:
    print(f"Successfully POSTED the resource. Status code: {response.status_code}.")
    print("=====")
    data = response.json()
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)
else:
    print(f"Failed to POST the resource. Status code: {response.status_code}.")

The response contents will be typical of many APIs that take POST requests, containing the ID of the newly-created resource (in this case, that resource is a new record for Guido van Rossum), the information you provided as POST data, and the date and time when the resource was created.

Note that in the code above, we checked for a response status code of `201`, not `200`. That’s because the `201` status code specifically indicates the successful creation of a new resource.

### Updating a Resource

There are two ways to update a resource in an API. The first way is to make a PUT request, which is for updating all the data in the resource, which requires you to provide all the data.

Suppose we want to update the entry for the user whose ID is 123 by overwriting it with new data: the creator of the Ada programming language, Jean Ichbiah:

In [ ]:
path_parameter = 123
url = f"{USERS_ENDPOINT}{path_parameter}"
put_data = {
    "language": "Ada",
    "creator":  "Jean Ichbiah",
}
response = requests.put(url, data=put_data)

if response.status_code == 200:
    print(f"Successfully PUT the resource. Status code: {response.status_code}.")
    print("=====")
    data = response.json()
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)
else:
    print(f"Failed to PUT the resource. Status code: {response.status_code}.")

The contents of the response contents will be typical for APIs that accept PUT requests. It will contain the all the data for the updated resource and the date and time when the update was made.

The other way to update a resource is to make a PATCH request, which is for updating only certain data in the resource, which requires you to provide only the data to be updated.

Suppose we want to update the entry we just updated so that the `language` property specifies that the Ada programming language is named after Ada Lovelace, who is famous for her work on Charles Babbage’s analytical engine. Do this by entering the code below into a new code cell and running it:

In [ ]:
path_parameter = 123
url = f"{USERS_ENDPOINT}{path_parameter}"
patch_data = {
    "language": "Ada (named after Ada Lovelace)",
}
response = requests.put(url, data=patch_data)

if response.status_code == 200:
    print(f"Successfully PATCHed the resource. Status code: {response.status_code}.")
    print("=====")
    data = response.json()
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)
else:
    print(f"Failed to PATCH the resource. Status code: {response.status_code}.")

The contents of the response contents will be typical for APIs that accept PATCH requests. It will contain the resource data that was updated and the date and time when the update was made.

### Deleting a Resource

Let’s delete the resource with the ID of `123` using the DELETE request:

In [ ]:
path_parameter = 123
url = f"{USERS_ENDPOINT}{path_parameter}"
response = requests.delete(url)

if response.status_code == 204:
    print(f"Successfully DELETEd the resource. Status code: {response.status_code}.")
else:
    print(f"Failed to DELETE the resource. Status code: {response.status_code}.")

Note that in the code above, we checked for a response status code of `204`, not `200`. That’s because the `204` status code specifically indicates the successful deletion of a resource.

In the next demo, you’ll add another way of calling APIs to your repertoire and then combine both ways to build a small but interesting AI-powered application.